In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import shutil
shutil.copytree('/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/training_data/conv_lstm/', '/content/training_data/')

In [ ]:
!pip install scikit-learn
!pip install joblib soundfile speechpy librosa matplotlib
!pip install joblib==1.2.0
!pip install np_utils

In [ ]:
#from include.functions import initialise, load_data
#from include.stats import plot_accuracy_loss
import tensorflow as tf
from tensorflow.keras.layers import Layer, Conv2D, Flatten, Dense
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
#from keras.utils import np_utils
import joblib
import os
#from keras.utils import np_utils
import sys
from keras import Sequential
from keras.layers import Dense, Dropout, LSTM, Conv1D, Flatten, MaxPooling1D
import matplotlib.pyplot as plt
###################################################
def load_data(load_from_file,name):
    if load_from_file:
        current_dir = os.getcwd()
        subdir = '\\training_data\\'
        save_dir = '/content/training_data'

        x_train = joblib.load(os.path.join(save_dir, 'x_train_'+name+'.joblib'))
        x_test = joblib.load(os.path.join(save_dir, 'x_test_'+name+'.joblib'))
        y_train = joblib.load(os.path.join(save_dir, 'y_train_'+name+'.joblib'))
        y_test = joblib.load(os.path.join(save_dir, 'y_test_'+name+'.joblib'))
        num_labels = joblib.load(os.path.join(save_dir, 'num_labels_'+name+'.joblib'))
        #data = joblib.load(os.path.join(save_dir, 'data_'+name+'.joblib'))
        #labels = joblib.load(os.path.join(save_dir, 'labels_'+name+'.joblib'))
    else:
        #x_train, x_test, y_train, y_test, num_labels = extract_data()
        current_dir = os.getcwd()
        subdir = '\\training_data\\'
        save_dir = current_dir + subdir
        joblib.dump(x_train, os.path.join(save_dir, 'x_train_'+name+'.joblib'))
        joblib.dump(x_test, os.path.join(save_dir, 'x_test_'+name+'.joblib'))
        joblib.dump(y_train, os.path.join(save_dir, 'y_train_'+name+'.joblib'))
        joblib.dump(y_test, os.path.join(save_dir, 'y_test_'+name+'.joblib'))
        joblib.dump(num_labels, os.path.join(save_dir, 'num_labels_'+name+'.joblib'))
        #joblib.dump(data, os.path.join(save_dir, 'data_'+name+'.joblib'))
        #joblib.dump(labels, os.path.join(save_dir, 'labels_'+name+'.joblib'))
    return x_train,x_test,y_train,y_test,num_labels#,data, labels
###################################################



def conv_lstm_example():
    print("*************START*************\n")
    #loadfromfile=False
    loadfromfile=True
    x_train,x_test,y_train,y_test,num_labels =load_data(load_from_file=loadfromfile,name= 'conv_lstm')

    print("Before Reshape")
    print("x_train.shape :" +  str(x_train.shape) +"    y_train.shape:"+ str(y_train.shape))
    print("x_test.shape :" +  str(x_test.shape) +"    y_test.shape:"+ str(y_test.shape))

    from keras import utils
    y_train = utils.to_categorical(y_train)
    y_test = utils.to_categorical(y_test)
    in_shape = x_train[0].shape

    #x_train = x_train.reshape(x_train.shape[0], in_shape[1], in_shape[0], 1)
    #x_test = x_test.reshape(x_test.shape[0], in_shape[1], in_shape[0], 1)

    print("After Reshape")
    print("x_train.shape :" +  str(x_train.shape) +"    y_train.shape:"+ str(y_train.shape))
    print("x_test.shape :" +  str(x_test.shape) +"    y_test.shape:"+ str(y_test.shape))

    num_timesteps, num_features = x_train.shape[1], x_train.shape[2]
    num_classes = y_train.shape[1]

    print("num_timesteps, num_features, num_classes")
    print("num_timesteps:" +str(num_timesteps) )
    print("num_features:" +str(num_features))
    print("num_classes:" +str(num_classes))

    #####################

    model = Sequential()
    model.add(Conv1D(128, kernel_size=5, activation='relu', input_shape=(num_timesteps, num_features)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.5))
    model.add(LSTM(64, return_sequences=True))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    #model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    #####################
     #####################
    loss_history = []
    accuracy_history = []

    epochs = 100

    for epoch in range(epochs):
        # Train the model for one epoch
        history = model.fit(x_train, y_train, epochs=1, validation_data=(x_test,y_test), verbose=0)

        # Record loss and accuracy values for plotting
        loss_history.append(history.history['loss'][0])
        accuracy_history.append(history.history['accuracy'][0])

        # Display the loss and accuracy for the current epoch
        print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss_history[-1]:.4f} - Accuracy: {accuracy_history[-1]:.4f}")

        # Plot loss and accuracy
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(loss_history)
        plt.title('Loss vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')

        plt.subplot(1, 2, 2)
        plt.plot(accuracy_history)
        plt.title('Accuracy vs. Epoch')
        plt.xlabel('Epoch')
        plt.ylabel('Accuracy')

        plt.tight_layout()
        plt.show()
    """
    history = model.fit(x_train, y_train,  batch_size=32, epochs=50,shuffle=True,validation_data=(x_test,y_test))

    loss, acc = model.evaluate(x_test, y_test)
    print("Accuracy: {:.2f}%".format(acc*100))
    print("Loss: {:.2f}%".format(loss*100))
    print("Highest Accuracy: {:.2f}%".format(max(history.history['val_accuracy'])*100))

    """
    import tensorflow as tf
    model.save('/content/drive/MyDrive/Personal/Study/Phd/Python/Examples/4.C-LSTM/model/conv_lstm_classifier.tf')
    #tf.saved_model.save(model, "model/conv_lstm_classifier")

    print('conv_lstm Done')


if __name__ == "__main__":
    conv_lstm_example()